In [0]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns

# Prepare data

In [0]:
# fix the random seed
seed = 42

# load the small conductor dataset here
path = '../data_files/sha/ecq_sha_B_100_conds_1_500000_reg.parquet'
columns = ['rank', 'regulator', 'torsion', 'sha', 'special_value', 'real_period', 'tamagawa_product']

# Read the specified columns using PyArrow
table = pq.read_table(path, columns=columns)
# Convert the PyArrow Table to a Pandas DataFrame
df = table.to_pandas()

# sqrt sha
df['sqrt_sha'] = df['sha'].apply(lambda x: int(x**0.5))
df.drop('sha', axis=1, inplace=True)

## Options for imbalanced dataset
- Option 1: downsize the majority class (sha trivial curves) 
- Option 2: upsize the minority class (sha nontrivial curves) 
- default is False to both

In [0]:
downsizing = False
upsizing = False

majority_ratio = 0.6
majority_ratio = 0.5

if downsizing == True:
    df_majority = df[df['sqrt_sha'] == 1]  # Filter out rows with sha == 1
    df_minority = df[df['sqrt_sha'] != 1]  # Filter out rows with sha != 1
    # Calculate the number of samples needed to make sha == 1 roughly 70% of the data
    target_majority_count = int(len(df_minority) / (1-majority_ratio) * majority_ratio)
    # Randomly sample the majority class to reduce it to the target count
    df_majority_downsampled = df_majority.sample(target_majority_count, random_state=seed)
    # Combine the downsampled majority class with the minority class
    df = pd.concat([df_majority_downsampled, df_minority])
    # Shuffle the resulting DataFrame to mix the rows
    df = df.sample(frac=1, random_state=seed).reset_index(drop=True)   
elif upsizing == True:
    minority_class = df[df['sqrt_sha'] > 1]
    majority_class = df[df['sqrt_sha'] == 1]
    # Resample the minority class
    minority_upsampled = resample(minority_class, replace=True, n_samples=int(len(majority_class)/majority_ratio*(1-majority_ratio)), random_state=seed)
    # Combine the upsampled minority class with the majority class
    df = pd.concat([majority_class, minority_upsampled])

# Train with conductor < 500k curves

In [0]:
# Run a tree regression model with rank and no regulator
# model is called model_w_rank

# Prepare the data
feature_columns = [c for c in df.columns if c != 'sqrt_sha']
feature_columns_no_reg = feature_columns.copy()
feature_columns_no_reg.remove('regulator')
X = df[feature_columns_no_reg]
y = df[['sqrt_sha']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# Train the model
model_w_rank = HistGradientBoostingRegressor(random_state=seed)
model_w_rank.fit(X_train, y_train)
y_pred = model_w_rank.predict(X_test)
y_pred_rounded_w_rank = np.round(y_pred).astype(int)

# # Convert y_test to a 1D list of actual values
y_test = y_test.values.flatten()

# Calculate accuracy
w_rank_accuracy = accuracy_score(y_pred_rounded_w_rank, y_test)

# calculate MCC
w_rank_MCC = matthews_corrcoef(y_test, y_pred_rounded_w_rank)

# get the unique predicted values 
y_test_less_500_unique = np.unique(y_test)
y_pred_less_500_unique_w_rank = np.unique(y_pred_rounded_w_rank)

In [0]:
# Run a tree-based regressor without rank or regulator
# model is called model_wo

# Prepare the data
feature_columns_no_rank = feature_columns.copy()
feature_columns_no_rank.remove('rank')
feature_columns_no_rank.remove('regulator')
X = df[feature_columns_no_rank]
y = df[['sqrt_sha']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# Train the model
model_wo = HistGradientBoostingRegressor(random_state=seed)
model_wo.fit(X_train, y_train)
y_pred = model_wo.predict(X_test)
y_pred_rounded_wo = np.round(y_pred).astype(int)

# # Convert y_test to a 1D list of actual values
y_test = y_test.values.flatten()

# Calculate accuracy
wo_accuracy = accuracy_score(y_pred_rounded_wo, y_test)

# calculate MCC
wo_MCC = matthews_corrcoef(y_test, y_pred_rounded_wo)

# get unique predicted values
y_pred_less_500_unique_wo = np.unique(y_pred_rounded_wo)

In [0]:
# Run a tree-based regressor without rank but with regulator
# model is called model_w_reg

# Prepare the data
feature_columns_no_rank = feature_columns.copy()
feature_columns_no_rank.remove('rank')
X = df[feature_columns_no_rank]
y = df[['sqrt_sha']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# Train the model
model_w_reg = HistGradientBoostingRegressor(random_state=seed)
model_w_reg.fit(X_train, y_train)
y_pred = model_w_reg.predict(X_test)
y_pred_rounded_w_reg = np.round(y_pred).astype(int)

# # Convert y_test to a 1D list of actual values
y_test = y_test.values.flatten()

# Calculate accuracy
w_reg_accuracy = accuracy_score(y_pred_rounded_w_reg, y_test)

# calculate MCC
w_reg_MCC = matthews_corrcoef(y_test, y_pred_rounded_w_reg)

# get unique predicted values
y_pred_less_500_unique_w_reg = np.unique(y_pred_rounded_w_reg)

In [0]:
# print the results
print('cond less than 500k curves: ')
print(f'unique sha values: {y_test_less_500_unique}')
print('-'*10)
print(f'Features with rank but no regulator gives acc: ', w_rank_accuracy)
print(f'Features without rank or regulator gives acc: ', wo_accuracy)
print(f'Features with regulator but no rank gives acc: ', w_reg_accuracy)
print('-'*10)
print(f'Features with rank but no regulator gives MCC: ', w_rank_MCC)
print(f'Features without rank or regulator gives MCC: ', wo_MCC)
print(f'Features with regulator but no rank gives MCC: ', w_reg_MCC)
print('-'*10)
print(f'unique pred values with rank but no regulator: {y_pred_less_500_unique_w_rank}')
print(f'unique pred values without rank or regulator: {y_pred_less_500_unique_wo}')
print(f'unique pred values with regulator but no rank: {y_pred_less_500_unique_w_reg}')

In [0]:
# check acc when sqrt > threshold 
threshold_acc = []
min_threshold = 1 # min threshold
max_threshold = 8 # max threshold

for threshold in range(min_threshold,max_threshold):
    y_test_gt_threshold_where = np.where(y_test >= threshold)
    y_test_gt_threshold = y_test[y_test_gt_threshold_where]
    y_pred_rounded_gt_threshold_w_rank = y_pred_rounded_w_rank[y_test_gt_threshold_where]
    y_pred_rounded_gt_threshold_wo = y_pred_rounded_wo[y_test_gt_threshold_where]
    y_pred_rounded_gt_threshold_w_reg = y_pred_rounded_w_reg[y_test_gt_threshold_where]
    threshold_acc.append([threshold, accuracy_score(y_pred_rounded_gt_threshold_w_rank, y_test_gt_threshold), accuracy_score(y_pred_rounded_gt_threshold_wo, y_test_gt_threshold), accuracy_score(y_pred_rounded_gt_threshold_w_reg, y_test_gt_threshold), len(y_test_gt_threshold)])
    
threshold_acc = pd.DataFrame(threshold_acc,index=[f'sqrt_sha (sha >= {i})' for i in range(min_threshold,max_threshold)],columns=['Threshold','with rank no reg','without rank or reg', 'with reg no rank', 'num of curves'])
threshold_acc

In [0]:
# plot threshold_acc
threshold_acc.drop(columns=['num of curves'],inplace=True)
threshold_acc = threshold_acc.melt(id_vars="Threshold", var_name="Features Type", value_name="Accuracy")

# Plotting
plt.figure(figsize=(10, 6))
sns.barplot(x="Threshold", y="Accuracy", hue="Features Type", data=threshold_acc)

# Labeling
plt.xlabel("Sqrt Sha Threshold")
plt.ylabel("Accuracy")
plt.title("Small Conductor Dataset Accuracy")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Test with >500k curves



In [0]:
path = '../data_files/sha/ecq_sha_B_1000_conds_lt_500k.parquet'

# Read the specified columns using PyArrow
table = pq.read_table(path, columns=columns)
# Convert the PyArrow Table to a Pandas DataFrame
df_large_cond = table.to_pandas()

# sqrt sha
df_large_cond['sqrt_sha'] = df_large_cond['sha'].apply(lambda x: int(x**0.5))
df_large_cond.drop('sha', axis=1, inplace=True)

# Prepare the data
label_col = 'sqrt_sha'
X_test_large_cond = df_large_cond.drop(columns=[label_col])
y_test_large_cond = df_large_cond[label_col].values

# Run the models on the large conductor dataset
# model_w_rank
X_test_large_cond_no_reg = X_test_large_cond.drop(columns = ['regulator'])
y_pred = model_w_rank.predict(X_test_large_cond_no_reg)
y_pred_w_rank = np.round(y_pred).astype(int)
y_pred_more_500_unique_w_rank = np.unique(y_pred_w_rank)

# model_wo
X_test_large_cond_no_reg.drop(columns = ['rank'],inplace=True)
y_pred = model_wo.predict(X_test_large_cond_no_reg)
y_pred_wo = np.round(y_pred).astype(int)
y_pred_more_500_unique_wo = np.unique(y_pred_wo)

# model_w_reg
X_test_large_cond.drop(columns = ['rank'],inplace = True)
y_pred = model_w_reg.predict(X_test_large_cond)
y_pred_w_reg = np.round(y_pred).astype(int)
y_pred_more_500_unique_w_reg = np.unique(y_pred_w_reg)

# # Convert y_test to a 1D list of actual values
y_test_large_cond = y_test_large_cond.flatten()

# Calculate accuracy
w_rank_accuracy = accuracy_score(y_test_large_cond, y_pred_w_rank)
wo_accuracy = accuracy_score(y_test_large_cond, y_pred_wo)
w_reg_accuracy = accuracy_score(y_test_large_cond, y_pred_w_reg)

# Calculate MCC
w_rank_MCC = matthews_corrcoef(y_test_large_cond, y_pred_w_rank)
wo_MCC = matthews_corrcoef(y_test_large_cond, y_pred_wo)
w_reg_MCC = matthews_corrcoef(y_test_large_cond, y_pred_w_reg)

In [0]:
print('cond more than 500k curves: ')
print(f'unique sha values: {np.unique(y_test_large_cond)}')
print('-'*10)
print(f'Features with rank but no regulator: gives acc: ', w_rank_accuracy)
print(f'Features without rank or regulator: gives acc: ', wo_accuracy)
print(f'Features with regulator but no rank: gives acc: ', w_reg_accuracy)
print('-'*10)
print(f'Features with rank but no regulator: gives MCC: ', w_rank_MCC)
print(f'Features without rank or regulator: gives MCC: ', wo_MCC)
print(f'Features with regulator but no rank: gives MCC: ', w_reg_MCC)
print('-'*10)
print(f'unique pred values with rank but no regulator: {y_pred_more_500_unique_w_rank}')
print(f'unique pred values without rank or regulator: {y_pred_more_500_unique_wo}')
print(f'unique pred values with regulator but no rank: {y_pred_more_500_unique_w_reg}')

In [0]:
# check acc when sqrt > threshold
threshold_acc = []

for threshold in range(1,8):
    y_test_large_cond_gt_threshold_where = np.where(y_test_large_cond >= threshold)[0]
    y_test_large_cond_gt_threshold = y_test_large_cond[y_test_large_cond_gt_threshold_where]
    
    y_pred_w_rank_gt_threshold = y_pred_w_rank[y_test_large_cond_gt_threshold_where]
    y_pred_wo_gt_threshold = y_pred_wo[y_test_large_cond_gt_threshold_where]
    y_pred_w_reg_gt_threshold = y_pred_w_reg[y_test_large_cond_gt_threshold_where]
    
    threshold_acc.append([threshold, accuracy_score(y_pred_w_rank_gt_threshold, y_test_large_cond_gt_threshold), accuracy_score(y_pred_wo_gt_threshold, y_test_large_cond_gt_threshold) , accuracy_score(y_pred_w_reg_gt_threshold, y_test_large_cond_gt_threshold), len(y_test_large_cond_gt_threshold_where)])
    
threshold_acc = pd.DataFrame(threshold_acc,index=[f'acc (sqrt_sha >= {i})' for i in range(1,8)],columns=['Threshold','with rank no reg','without rank or reg', 'with reg no rank', 'num of curves'])
threshold_acc

In [0]:
# plot threshold_acc
threshold_acc.drop(columns=['num of curves'],inplace=True)
threshold_acc = threshold_acc.melt(id_vars="Threshold", var_name="Features Type", value_name="Accuracy")

# Plotting
plt.figure(figsize=(10, 6))
sns.barplot(x="Threshold", y="Accuracy", hue="Features Type", data=threshold_acc)

# Labeling
plt.xlabel("Sqrt Sha Threshold")
plt.ylabel("Accuracy")
plt.title("Large Conductor Dataset Accuracy")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Appendix: plotting curves with trivial sha

In [0]:
# for small conductor dataset

rank0_proportion = []
for i in range(len(df['sqrt_sha'].unique())+1):
    df_gt = df[df['sqrt_sha'] >= i]
    rank0_proportion.append(len(df_gt[df_gt['rank'] == 0])/len(df_gt) * 100)
    
# plot the proportion of rank 0 curves as a function of sqrt_sha using seaborn
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")
plt.figure(figsize=(12,6))
sns.lineplot(x=range(len(df['sqrt_sha'].unique())+1), y=rank0_proportion)
plt.xlabel('lower bound of sqrt_sha')
plt.ylabel('Proportion of rank 0 curves (%)')
plt.title('Proportion of rank 0 curves (%) in the small conductor dataset')

In [0]:
# for big conductor dataset

rank0_proportion = []
for i in range(len(df_large_cond['sqrt_sha'].unique())+1):
    df_gt = df_large_cond[df_large_cond['sqrt_sha'] >= i]
    rank0_proportion.append(len(df_gt[df_gt['rank'] == 0])/len(df_gt) * 100)
    
# plot the proportion of rank 0 curves as a function of sqrt_sha using seaborn
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")
plt.figure(figsize=(12,6))
sns.lineplot(x=range(len(df_large_cond['sqrt_sha'].unique())+1), y=rank0_proportion)
plt.xlabel('sqrt_sha threshold')
plt.ylabel('Proportion of rank 0 curves (%)')